In [ ]:
# !pip install python-dotenv langchain-core langchain-openai langchain-community

# Setup the demo

In [1]:
import json
import os
import requests

from bs4 import BeautifulSoup, Comment, Tag
from dotenv import load_dotenv

from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

In [26]:
load_dotenv("../.env")

True

In [16]:
system_message = (
    "You are given HTML elements of a German news website."
    "The elements do not only contain the content of the news articles but also some additional elements "
    "such as menu items, tables, dates, authors, etc. "
    "The objective is to extract only the relevant news article content and ignore the rest. "
    "Return a single string containing the result and keep the result in German."
    "Limit your response to at most five paragraphs."
)


def extract_news_article_text(html_input, llm):
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_message),
            ("human", "{input}")
        ]
    )
    chain = prompt | llm
    response = chain.invoke({"input": html_input})
    return response


api_cost = {
    "gpt-4o": {
        "prompt_tokens": 0.01,
        "completion_tokens": 0.03,
    },
}

def calculate_cost(llm_response, model_name):
    token_usage = llm_response.response_metadata["token_usage"]
    prompt_tokens = token_usage["prompt_tokens"]
    prompt_tokens_cost = api_cost[model_name]["prompt_tokens"]
    completion_tokens = token_usage["completion_tokens"]
    completion_tokens_cost = api_cost[model_name]["completion_tokens"]
    
    total_cost = (prompt_tokens * prompt_tokens_cost + completion_tokens * completion_tokens_cost) / 1000
    return total_cost

In [17]:
llm = ChatOpenAI(model="gpt-4o", temperature=0, api_key=os.getenv("OPENAI_API_KEY"))

# Example 1

[Link](https://www.handelsblatt.com/finanzen/banken-versicherungen/banken/ukraine-krieg-grossbanken-im-euroraum-haben-russland-engagements-deutlich-verringert-01/100037514.html) to the news article.

In [52]:
html_content = json.load(open("../data/cleaned_sites/100037514.html"))
print(html_content)

<html>
 <body>
  <app-root>
   <div>
    <div>
     <span>
      Menü
     </span>
     <span>
      Anmelden
     </span>
    </div>
    <article>
     <app-story-detail-page-header>
      <div>
       <app-header-content-kicker>
        <span>
         Ukraine-Krieg
        </span>
       </app-header-content-kicker>
       <app-header-content-headline>
        <h2>
         Großbanken im Euroraum haben Russland-Engagements deutlich verringert
        </h2>
       </app-header-content-headline>
       <app-header-content-lead-text>
        Die Banken der Euro-Zone haben ihr Engagement in Russland stark gesenkt. Die Europäische Zentralbank hatte den Rückgang gefordert. Nun macht aber ein anderer Bereich Sorgen.
       </app-header-content-lead-text>
       <app-story-date>
        15.05.2024 - 11:30 Uhr
       </app-story-date>
      </div>
     </app-story-detail-page-header>
     <app-storyline-elements>
      <app-storyline-element>
       <p>
        <span>
         Frankfurt.
   

In [53]:
result = extract_news_article_text(html_content, llm)
print(result.content)

Ukraine-Krieg

Großbanken im Euroraum haben Russland-Engagements deutlich verringert

Die Banken der Euro-Zone haben ihr Engagement in Russland stark gesenkt. Die Europäische Zentralbank hatte den Rückgang gefordert. Nun macht aber ein anderer Bereich Sorgen.

Frankfurt. Große Geldhäuser aus Ländern der Euro-Zone haben ihre Engagements in Russland laut EZB um mehr als die Hälfte verringert. Die Europäische Zentralbank (EZB) habe die Präsenz der Großinstitute dort genau überwacht, erklärte EZB-Chefbankenaufseherin Claudia Buch am Montag in einer Mitteilung zu einem Treffen der Euro-Finanzminister in Brüssel.

„Die Engagements scheinen sich ziemlich in Grenzen zu halten, und die direkten Engagements sind zwischen Ende 2021 und Ende 2023 um 55 Prozent zurückgegangen“, führte sie aus. Einige Banken hätten sich vollständig aus dem russischen Markt zurückgezogen.


In [54]:
result.response_metadata["token_usage"]

{'completion_tokens': 196, 'prompt_tokens': 564, 'total_tokens': 760}

In [55]:
calculate_cost(result, "gpt-4o")

0.011519999999999999

# Example 2

[Link](https://www.handelsblatt.com/finanzen/banken-versicherungen/banken/immobilien-pbb-soll-riesenportfolio-verkauf-mit-blackstone-verhandeln-01/100037533.html) to the news article.

In [56]:
html_content = json.load(open("../data/cleaned_sites/100037533.html"))
print(html_content)

<html>
 <body>
  <app-root>
   <div>
    <div>
     <span>
      Menü
     </span>
     <span>
      Anmelden
     </span>
    </div>
    <article>
     <app-story-detail-page-header>
      <div>
       <app-header-content-kicker>
        <span>
         Immobilien
        </span>
       </app-header-content-kicker>
       <app-header-content-headline>
        <h2>
         PBB soll Riesenportfolio-Verkauf mit Blackstone verhandeln
        </h2>
       </app-header-content-headline>
       <app-header-content-lead-text>
        Die Bank verbucht im ersten Quartal trotz der Krise bei Büroimmobilien einen Gewinn. Sie steht kurz davor, ein großes Portfolio in den USA und Großbritannien zu verkaufen.
       </app-header-content-lead-text>
       <app-header-content-authors>
        <app-authors-section>
         <app-authors>
          <a>
           Elisabeth Atzler
          </a>
         </app-authors>
         <app-story-date>
          15.05.2024 - 10:30 Uhr
         </app-story-date>

In [61]:
result = extract_news_article_text(html_content, llm)
print(result.content)

PBB soll Riesenportfolio-Verkauf mit Blackstone verhandeln. Die Bank verbucht im ersten Quartal trotz der Krise bei Büroimmobilien einen Gewinn. Sie steht kurz davor, ein großes Portfolio in den USA und Großbritannien zu verkaufen.

Frankfurt. Kurz nach Amtsantritt des neuen Vorstandschefs erzielt der Immobilienfinanzierer Deutsche Pfandbriefbank (PBB) wieder einen Gewinn. Trotz der anhaltenden Krise bei Büro- und Gewerbeimmobilien verdiente die Bank im ersten Quartal 2024 vor Steuern 34 Millionen Euro, was leicht über dem Vorjahreswert liegt. „Wir sind trotz volatiler Märkte gut in das Jahr 2024 gestartet“, sagte PBB-Chef Kay Wolf am Dienstag.

Der frühere Manager der Deutschen Bank hat den Posten im März von Andreas Arndt übernommen. Der Wechsel war seit dem vergangenen Herbst bekannt gewesen. Das solide Ergebnis unterstreiche, „dass die PBB eine im Kern gesunde und nachhaltig profitable Bank ist“, sagte Wolf. Die PBB-Aktie stieg am Dienstagnachmittag um sieben Prozent und setzte dam

In [62]:
result.response_metadata["token_usage"]

{'completion_tokens': 246, 'prompt_tokens': 677, 'total_tokens': 923}

In [63]:
calculate_cost(result, "gpt-4o")

0.014150000000000001

# Example 3

[Link](https://www.handelsblatt.com/finanzen/banken-versicherungen/banken/finanzbranche-aareal-bank-muss-sich-neuen-finanzvorstand-suchen-ergebnis-steigt/100037723.html) to the news article.

In [64]:
html_content = json.load(open("../data/cleaned_sites/100037723.html"))
print(html_content)

<html>
 <body>
  <app-root>
   <div>
    <div>
     <span>
      Menü
     </span>
     <span>
      Anmelden
     </span>
    </div>
    <article>
     <app-story-detail-page-header>
      <div>
       <app-header-content-kicker>
        <span>
         Finanzbranche
        </span>
       </app-header-content-kicker>
       <app-header-content-headline>
        <h2>
         Aareal Bank muss sich neuen Finanzvorstand suchen – Ergebnis steigt
        </h2>
       </app-header-content-headline>
       <app-header-content-lead-text>
        Der Immobilienfinanzierer bekommt nach der Übernahme durch Investoren einen neuen Finanzvorstand. Risikokredite sortiert die Bank derweil weiter aus.
       </app-header-content-lead-text>
       <app-story-date>
        15.05.2024 - 09:29 Uhr
       </app-story-date>
      </div>
     </app-story-detail-page-header>
     <app-storyline-elements>
      <app-storyline-element>
       <p>
        <span>
         Frankfurt.
        </span>
        Nach 

In [65]:
result = extract_news_article_text(html_content, llm)
print(result.content)

Finanzbranche

Aareal Bank muss sich neuen Finanzvorstand suchen – Ergebnis steigt

Der Immobilienfinanzierer bekommt nach der Übernahme durch Investoren einen neuen Finanzvorstand. Risikokredite sortiert die Bank derweil weiter aus.

Frankfurt. Nach der Übernahme durch Finanzinvestoren bekommt der Immobilienfinanzierer Aareal Bank auch einen neuen Finanzvorstand. Marc Heß werde Ende des Jahres, wenn sein Vertrag auslaufe, aus dem Unternehmen ausscheiden, teilte das Wiesbadener Unternehmen am Mittwoch mit. Über seine Nachfolge werde zeitnah entschieden.

Ende Juli wechselt bereits der Vorstandschef: Jochen Klösges verlässt die Aareal Bank, sein Amt übernimmt Christian Ricken, bisher Vorstand bei der LBBW. Im vergangenen Jahr hatten die Finanzinvestoren Advent und Centerbridge sowie der kanadische Pensionsfonds CPPIB das Institut übernommen.


In [66]:
result.response_metadata["token_usage"]

{'completion_tokens': 186, 'prompt_tokens': 556, 'total_tokens': 742}

In [67]:
calculate_cost(result, "gpt-4o")

0.01114

## Example 4

[Link](https://www.handelsblatt.com/finanzen/banken-versicherungen/banken/grossbank-citigroup-muss-fuer-irrtuemlichen-aktienverkauf-millionen-zahlen/100039229.html) to the news article.

We demonstrate cleaning the article fully and then extracting the content through GPT-4o.

In [6]:
def clean_html(soup_element):
    if isinstance(soup_element, Comment):
        soup_element.extract()
        return
    if hasattr(soup_element, 'find_all'):
        # Remove style and script tags
        blacklisted_elements = ['head', 'style', 'script', 'footer', 'nav', 'noscript', 'table']
        for element in soup_element.find_all(blacklisted_elements):
            element.decompose()

        # Additionally remove elements containing 'nav' or 'menu' in their tag name
        blacklisted_patterns = [
            'nav', 'menu', 'image', 'img', 'button', 'btn', 'footer', 'table', 'breadcrumb', 'swipe', 'slide',
            'video', 'banner',
        ]
        for element in soup_element.find_all(lambda tag: any([x in tag.name for x in blacklisted_patterns])):
            element.decompose()

        # Remove comments
        for comment in soup_element.find_all(string=lambda text: isinstance(text, Comment)):
            comment.extract()

        # Remove all attributes
        if hasattr(soup_element, 'attrs'):
            soup_element.attrs = {}

        # Process children first
        children = list(soup_element.children)
        for child in children:
            clean_html(child)

        # Remove unnecessary nesting
        if len(soup_element.contents) == 1 and isinstance(soup_element.contents[0], Tag):
            child = soup_element.contents[0]
            if not child.attrs and len(child.contents) == 1 and isinstance(child.contents[0], Tag):
                grandchild = child.contents[0]
                if grandchild.text.strip():
                    soup_element.replace_with(grandchild)
                else:
                    soup_element.replace_with(child)

        # Remove empty elements after children are processed
        if soup_element.name:
            # Check if element has no text and no non-empty children
            non_empty_children = [child for child in soup_element.children if
                                  not (isinstance(child, str) and not child.strip())]
            if not soup_element.text.strip() and not non_empty_children:
                soup_element.decompose()

In [34]:
with open("../data/full_sites/100039229.html", "r") as f:
    html_content = f.read()

In [35]:
soup = BeautifulSoup(html_content, 'html.parser')
clean_html(soup)
cleaned_html_content = soup.prettify()

print(cleaned_html_content)

<html>
 <body>
  <app-root>
   <div>
    <div>
     <span>
      Menü
     </span>
     <span>
      Anmelden
     </span>
    </div>
    <article>
     <app-story-detail-page-header>
      <div>
       <app-header-content-kicker>
        <span>
         Großbank
        </span>
       </app-header-content-kicker>
       <app-header-content-headline>
        <h2>
         Citigroup muss für irrtümlichen Aktienverkauf Millionen zahlen
        </h2>
       </app-header-content-headline>
       <app-header-content-lead-text>
        Ein Eingabefehler am Computer hatte kurzzeitig ein Chaos an mehreren Börsen ausgelöst. Jetzt muss die verantwortliche Bank dafür eine hohe Strafe zahlen.
       </app-header-content-lead-text>
       <app-story-date>
        22.05.2024 - 15:27 Uhr
       </app-story-date>
      </div>
     </app-story-detail-page-header>
     <app-storyline-elements>
      <app-storyline-element>
       <app-storyline-elements-aside-wrapper>
        <p>
         <span>
       

In [20]:
result = extract_news_article_text(cleaned_html_content, llm)
print(result.content)

Citigroup muss für irrtümlichen Aktienverkauf Millionen zahlen

Ein Eingabefehler am Computer hatte kurzzeitig ein Chaos an mehreren Börsen ausgelöst. Jetzt muss die verantwortliche Bank dafür eine hohe Strafe zahlen.

London. Die US-Großbank Citigroup muss wegen irrtümlicher Aktienverkäufe in Großbritannien eine Millionenstrafe zahlen. Der Fehler eines Aktienhändlers hatte im Jahr 2022 zum ungewollten Verkauf von Papieren im Wert von 1,4 Milliarden US-Dollar (1,3 Mrd Euro) geführt. Die internen Kontrollen der Bank hätten dies nicht verhindert, teilte die britische Finanzmarktaufsicht FCA am Mittwoch mit. Sie bemängelte die Systeme und die Kontrollmechanismen des Instituts.

Insgesamt bezahlt die Citigroup nun gut 61,6 Millionen britische Pfund (gut 72 Millionen Euro) an Strafen. Die Summe setzt sich aus zwei Teilen zusammen – einer Strafe der FCA selbst und einer weiteren, die die Finanzdienstleistungsaufsicht PRA verhängte.

Ein Aktienhändler hatte den Schaden ausgelöst. Er wollte ei

In [21]:
result.response_metadata["token_usage"]

{'completion_tokens': 428, 'prompt_tokens': 880, 'total_tokens': 1308}

In [22]:
calculate_cost(result, "gpt-4o")

0.02164

## Example 5

[Link](https://www.handelsblatt.com/finanzen/banken-versicherungen/banken/grossbank-citigroup-muss-fuer-irrtuemlichen-aktienverkauf-millionen-zahlen/100039229.html) to the news article.

In [36]:
%%time

url = 'https://www.handelsblatt.com/finanzen/banken-versicherungen/banken/grossbank-citigroup-muss-fuer-irrtuemlichen-aktienverkauf-millionen-zahlen/100039229.html'
apikey = os.getenv("ZENROWS_API_KEY")
params = {
    'url': url,
    'apikey': apikey,
	'js_render': 'true',
	'js_instructions': """[{"click":".selector"},{"wait":500},{"fill":[".input","value"]},{"wait_for":".slow_selector"}]""",
	'premium_proxy': 'true',
	'proxy_country': 'de',
}
response = requests.get('https://api.zenrows.com/v1/', params=params)
html_content = response.text
len(html_content)

CPU times: user 18.7 ms, sys: 10.6 ms, total: 29.3 ms
Wall time: 20.2 s


381724

In [37]:
soup = BeautifulSoup(html_content, 'html.parser')
clean_html(soup)
cleaned_html_content = soup.prettify()

print(cleaned_html_content)

<html>
 <body>
  <app-root>
   <div>
    <div>
     <span>
      Menü
     </span>
     <span>
      Anmelden
     </span>
    </div>
    <article>
     <app-story-detail-page-header>
      <div>
       <app-header-content-kicker>
        <span>
         Großbank
        </span>
       </app-header-content-kicker>
       <app-header-content-headline>
        <h2>
         Citigroup muss für irrtümlichen Aktienverkauf Millionen zahlen
        </h2>
       </app-header-content-headline>
       <app-header-content-lead-text>
        Ein Eingabefehler am Computer hatte kurzzeitig ein Chaos an mehreren Börsen ausgelöst. Jetzt muss die verantwortliche Bank dafür eine hohe Strafe zahlen.
       </app-header-content-lead-text>
       <app-story-date>
        22.05.2024 - 15:27 Uhr
       </app-story-date>
      </div>
     </app-story-detail-page-header>
     <app-storyline-elements>
      <app-storyline-element>
       <app-storyline-elements-aside-wrapper>
        <p>
         <span>
       

In [31]:
result = extract_news_article_text(cleaned_html_content, llm)
print(result.content)

Citigroup muss für irrtümlichen Aktienverkauf Millionen zahlen

Ein Eingabefehler am Computer hatte kurzzeitig ein Chaos an mehreren Börsen ausgelöst. Jetzt muss die verantwortliche Bank dafür eine hohe Strafe zahlen.

London. Die US-Großbank Citigroup muss wegen irrtümlicher Aktienverkäufe in Großbritannien eine Millionenstrafe zahlen. Der Fehler eines Aktienhändlers hatte im Jahr 2022 zum ungewollten Verkauf von Papieren im Wert von 1,4 Milliarden US-Dollar (1,3 Mrd Euro) geführt. Die internen Kontrollen der Bank hätten dies nicht verhindert, teilte die britische Finanzmarktaufsicht FCA am Mittwoch mit. Sie bemängelte die Systeme und die Kontrollmechanismen des Instituts.

Insgesamt bezahlt die Citigroup nun gut 61,6 Millionen britische Pfund (gut 72 Millionen Euro) an Strafen. Die Summe setzt sich aus zwei Teilen zusammen – einer Strafe der FCA selbst und einer weiteren, die die Finanzdienstleistungsaufsicht PRA verhängte.

Ein Aktienhändler hatte den Schaden ausgelöst. Er wollte ei

In [32]:
result.response_metadata["token_usage"]

{'completion_tokens': 378, 'prompt_tokens': 880, 'total_tokens': 1258}

In [33]:
calculate_cost(result, "gpt-4o")

0.02014